### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown
import os

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o-mini",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
            api_version="2024-12-01-preview"
        )
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o-mini",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
            api_version="2024-12-01-preview"
        )
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o-mini",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
            api_version="2024-12-01-preview"
        )
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen for your new AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen excels in enabling multiple AI agents to collaborate dynamically. This is particularly beneficial for complex tasks that require decision-making and iterative interaction.

2. **Simplified Development**: It provides a framework that simplifies the orchestration, optimization, and automation of large language model (LLM) workflows, allowing developers to focus more on the strategic implementation rather than the technical intricacies.

3. **Customizable Agents**: AutoGen allows for the creation of customizable, conversation-driven agents that can adapt to various tasks, making it flexible for diverse applications.

4. **Autonomous Functionality**: The framework supports agents that can autonomously solve problems, generate code, and perform complex tasks using built-in tools like code executors and function callers.

5. **Strong Backing**: With support from Microsoft, AutoGen has a reliable foundation, implying ongoing improvements and a solid community for knowledge sharing, which can be advantageous during development.

6. **Focused on Agentic AI**: AutoGen specifically targets the development of agentic AI, providing an easy-to-use environment that accelerates research and applications in this innovative field.

Choosing AutoGen could enhance your project by leveraging these strengths to create efficient and effective AI agents. 

TERMINATE

## Cons of AutoGen:
Here are several cons associated with the AutoGen AI framework that you may want to consider for your project:

1. **Steeper Learning Curve**: AutoGen has a reputation for being more complex due to its open-source nature, which may require developers to invest more time in learning how to use it effectively.

2. **Limited User Interface**: The framework is primarily CLI-first, which means that it lacks robust graphical user interface options, potentially making it less accessible for teams that prefer more visually oriented tools.

3. **Customization Challenges**: While it offers fine-tuned control, this can translate into less flexibility for simpler use cases. Users looking for straightforward implementations might find the customization requirements burdensome.

4. **Deployment Complexity**: Deploying AutoGen typically requires significant customization, which might complicate the deployment process compared to other frameworks that offer easier out-of-the-box solutions.

5. **Infrastructure Management**: Users will need to manage their hosting and infrastructure, which adds an additional layer of complexity, particularly for teams without strong DevOps capabilities.

6. **Less Ideal for Complex Automations**: Although powerful in many respects, AutoGen may not be the best choice for projects needing highly sophisticated AI-driven automations.

These factors could influence your decision on whether AutoGen is the right fit for your AI Agent project. 

TERMINATE



## Decision:

Based on the pros and cons presented, I recommend using AutoGen for the project.

**Rationale**: The strengths of AutoGen, particularly in enabling multi-agent collaboration, simplifying development, and promoting autonomous functionality, align well with the goals of creating effective and efficient AI agents. While there are concerns regarding its complexity and deployment challenges, the compelling advantages—such as strong support from Microsoft and a focus on agentic AI—suggest that the long-term benefits will outweigh the initial learning curve and implementation hurdles. The flexibility and customization options available can be leveraged to build a robust solution suited to our project's needs.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()